# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [145]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [146]:
# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creates a list of files that will be used to extract data
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [147]:
full_data_rows_list = [] 

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)    
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    
    #Writes header
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        #Skips rows without an artist
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### Creating a Cluster

In [148]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [149]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS musical_events
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

#### Set Keyspace

In [150]:
session.set_keyspace('musical_events')

## The following tables are set up to answer the following three example queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [151]:
query = """
    CREATE TABLE IF NOT EXISTS artist_library (
        itemInSession int,
        sessionId int,
        artist text,
        song_title text,
        song_length double,
        PRIMARY KEY(itemInSession, sessionId))
"""
session.execute(query)
                    

In [152]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (itemInSession, sessionId, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

In [153]:
rows = session.execute("""
    SELECT
        artist,
        song_title,
        song_length
    FROM
        artist_library
    WHERE itemInSession=4 AND sessionId=338
""")

for row in rows:
    print(row)

In [154]:
query = """
    CREATE TABLE IF NOT EXISTS artist_user_lib (
        user_id int,
        session_id int,
        itemInSession int,
        artist text,
        song_title text,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY((user_id, session_id), itemInSession)
    ) WITH CLUSTERING ORDER BY (itemInSession ASC)
"""
session.execute(query)
                    

In [155]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = """
            INSERT INTO artist_user_lib (
                user_id,
                session_id,
                itemInSession,
                artist,
                song_title,
                user_first_name,
                user_last_name
            )
        """     
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [156]:
rows = session.execute("""
    SELECT
        artist,
        song_title,
        user_first_name,
        user_last_name
    FROM
        artist_user_lib
    WHERE user_id=10 AND session_id=182
""")

for row in rows:
    print(row)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


In [157]:
query = """
    CREATE TABLE IF NOT EXISTS user_library (
        song_title text,
        user_id int,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY(song_title, user_id))
"""
session.execute(query)
                    

In [158]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_library (song_title, user_id, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [159]:
rows = session.execute("""
    SELECT
        user_first_name,
        user_last_name
    FROM
        user_library
    WHERE song_title='All Hands Against His Own'
""")

for row in rows:
    print(row)

### Drop the tables before closing out the sessions

In [160]:
## TO-DO: Drop the table before closing out the sessions
drop_query = """DROP TABLE IF EXISTS {}"""

session.execute(drop_query.format('artist_library'))
session.execute(drop_query.format('artist_user_lib'))
session.execute(drop_query.format('user_library'))

### Close the session and cluster connection¶

In [161]:
session.shutdown()
cluster.shutdown()